In [4]:
import os
os.environ["HF_HUB_DISABLE_SYMLINKS_WARNING"] = "1"

In [ ]:
# Import libraries
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments, DataCollatorWithPadding
from datasets import Dataset
import numpy as np
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

In [ ]:
# Step 1: Load and preprocess data
# โหลดข้อมูลและรวมชุดข้อมูลการฝึก
train_data_part1 = pd.read_csv('TrainData01.csv')
train_data_part2 = pd.read_csv('TrainData02.csv')
train_data = pd.concat([train_data_part1, train_data_part2], ignore_index=True)

test_data = pd.read_csv('TestData.csv')

In [ ]:
# รวมข้อความจากหลายคอลัมน์เพื่อใช้เป็นฟีเจอร์ข้อความ
train_data['text'] = train_data['subject'] + ' ' + train_data['email_from'] + ' ' + train_data['email_to'] + ' ' + train_data['cleanMessage']
test_data['text'] = test_data['subject'] + ' ' + test_data['email_from'] + ' ' + test_data['email_to'] + ' ' + test_data['cleanMessage']

In [ ]:
# ลบข้อมูลที่มีค่า NaN
train_data = train_data.dropna(subset=['text', 'label'])
test_data = test_data.dropna(subset=['text', 'label'])

In [ ]:
# แปลงข้อมูลเป็น Huggingface Dataset
train_dataset = Dataset.from_pandas(train_data[['text', 'label']])
test_dataset = Dataset.from_pandas(test_data[['text', 'label']])

In [ ]:
# Step 2: Tokenization
# โหลด tokenizer สำหรับ DistilBERT
model_name = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [ ]:
# สร้างฟังก์ชันสำหรับ Tokenize
def tokenize_function(examples):
    return tokenizer(examples['text'], padding="max_length", truncation=True)

In [ ]:
# ใช้ Tokenizer กับข้อมูล
tokenized_train_dataset = train_dataset.map(tokenize_function, batched=True)
tokenized_test_dataset = test_dataset.map(tokenize_function, batched=True)

In [ ]:
# Step 3: Prepare the model
# โหลดโมเดล DistilBERT สำหรับการจำแนกประเภท
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

In [ ]:
# Step 4: Training arguments
# ตั้งค่าพารามิเตอร์สำหรับการฝึกโมเดล
training_args = TrainingArguments(
    output_dir="./results",                 # ไดเรกทอรีสำหรับบันทึกผล
    eval_strategy="epoch",                 # ประเมินผลทุกๆ 1 epoch
    save_strategy="epoch",                 # บันทึกโมเดลทุกๆ 1 epoch
    learning_rate=2e-5,                    # Learning rate
    per_device_train_batch_size=16,        # Batch size สำหรับการฝึก
    per_device_eval_batch_size=16,         # Batch size สำหรับการประเมิน
    num_train_epochs=3,                    # จำนวนรอบการฝึก
    weight_decay=0.01,                     # ค่า Weight Decay
    logging_dir="./logs",                  # ไดเรกทอรีสำหรับ Logs
    logging_steps=10,                      # Log ทุกๆ 10 step
    load_best_model_at_end=True,           # โหลดโมเดลที่ดีที่สุดเมื่อสิ้นสุดการฝึก
    metric_for_best_model="accuracy"       # ใช้ Accuracy ในการวัดโมเดลที่ดีที่สุด
)

In [ ]:
# Step 5: Data Collator
# ใช้ DataCollatorWithPadding แทน tokenizer ใน Trainer
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [ ]:
# Step 6: Define metrics
# ฟังก์ชันคำนวณ metrics สำหรับการประเมินผล
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average='binary')
    acc = accuracy_score(labels, predictions)
    return {"accuracy": acc, "precision": precision, "recall": recall, "f1": f1}

In [ ]:
# Step 7: Create Trainer
# สร้าง Trainer สำหรับการฝึกและประเมินโมเดล
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_test_dataset,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

In [ ]:
# Step 8: Train the model
print("Starting model training...")
trainer.train()

In [ ]:
# Step 9: Evaluate the model
print("Evaluating the model...")
results = trainer.evaluate()
print("Evaluation Results:", results)

In [ ]:
# Step 10: Save the model
# บันทึกโมเดลและ tokenizer
print("Saving the model...")
model.save_pretrained('./phishing_detector_model')
tokenizer.save_pretrained('./phishing_detector_model')

print("Training and evaluation completed successfully!")